In [12]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage.io
import os
import coco
import yolo
import vgg

In [2]:
repo_dir = project_name = 'COTSDataset' 
part1_single_objects = os.path.join(repo_dir, "Part 1 - Single Objects")
part2_multiple_objects = os.path.join(repo_dir, "Part 2 - Multiple Objects")
part3_complex_background = os.path.join(repo_dir, "Part 3 - Complex Background")

In [3]:
do_display = True
do_save = False
do_print = False

# COCO
coco_output_dir = "COCO_annotations"
coco_color = (0, 255, 0)

# YOLO
yolo_output_dir = "YOLO_annotations"
yolo_color = (0, 255, 0)

# VGG
vgg_output_dir = "VGG_annotations"
vgg_color = 'r'

In [4]:
def iou(annotation):
    gt = annotation['image']
    annotation_mask = np.zeros((annotation['height'], annotation['width']), dtype=np.uint8)
    cv2.drawContours(annotation_mask, annotation['contours'], -1, 255, -1)
    # resize annotation mask to gt size
    annotation_mask = cv2.resize(annotation_mask, (gt.shape[1], gt.shape[0]))

    # plot both on each other transparently, one blue one red
    # plt.figure()
    # plt.rcParams['figure.figsize'] = [20, 10]
    # plt.imshow(gt, cmap='Blues', alpha=0.5)
    # plt.imshow(annotation_mask, cmap='Reds', alpha=0.5)
    # plt.xticks([])
    # plt.yticks([])
    # plt.show()

    intersection = np.logical_and(gt, annotation_mask)
    union = np.logical_or(gt, annotation_mask)
    return np.sum(intersection) / np.sum(union)

In [5]:
# Import coco
from pycocotools.coco import COCO
from pycocotools import mask as maskUtils
import skimage.io as io

In [7]:
ann_file = 'Evaluation/instances_train2017.json'
coco_data = COCO(ann_file)

loading annotations into memory...
Done (t=15.65s)
creating index...
index created!


In [8]:
coco_annotations = coco_data.loadAnns(coco_data.getAnnIds())

In [ ]:
# coco_annotations = coco_data.loadAnns(coco_data.getAnnIds())

# # Merge annotations with same image id
# merged_annotations = {}
# for annotation in coco_annotations:
#     # print(merged_annotations)
#     if annotation['image_id'] in merged_annotations:
#         merged_annotations[annotation['image_id']]['segmentation'].extend(annotation['segmentation'])
#     else:
#         merged_annotations[annotation['image_id']] = annotation

In [9]:
from random import randint
rand_id = randint(0, len(coco_annotations))

In [10]:
import skimage.io

In [24]:
rand_id = 555669

In [25]:
id_ = coco_annotations[rand_id]['image_id']
name = coco_data.loadImgs(id_)[0]['file_name']
image = coco_data.annToMask(coco_annotations[rand_id])

original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])


im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

annotation = coco.annotate(im, do_display=True, do_save=False, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.POLY_APPROX)

contours = annotation['contours']

annotated_image = np.zeros((annotation['height'], annotation['width'], 3), dtype=np.uint8)
cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)


 Annotating image:  000000034381.jpg
------------------------------------------------------------------------------------------------------------------------


c:\Work\uni\Summer Placement\mask-to-annotation\coco.py:60: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [16]:
import matplotlib.pyplot as plt

In [26]:
original_mask = image

plt.figure()
plt.rcParams['figure.figsize'] = [20, 10]
plt.subplot(131)
plt.imshow(original)
plt.xticks([])
plt.yticks([])
plt.title('Original Image', fontsize=20)
plt.subplot(132)
image_rgba = cv2.cvtColor(original_mask, cv2.COLOR_GRAY2RGBA)
alpha = np.where(image == 0, 0, 255)
image_rgba[:, :, 0] = 0
image_rgba[:, :, 1] = 0
image_rgba[:, :, 2] = 255 
image_rgba[:, :, 3] = alpha

plt.imshow(original, alpha=1)
plt.imshow(image_rgba, alpha=0.5)

plt.xticks([])
plt.yticks([])
plt.title('Original Mask', fontsize=20)
plt.subplot(133)
# plot contours
plt.imshow(original_mask, cmap='gray')
# plt.imshow(annotated_image)
annotated_rgba = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGBA)
alpha = np.where(annotated_image == 0, 0, 255)
annotated_rgba[:, :, 0] = 255
annotated_rgba[:, :, 1] = 0
annotated_rgba[:, :, 2] = 0
annotated_rgba[:, :, 3] = alpha[:, :, 0]
plt.imshow(annotated_rgba, alpha=1)


plt.xticks([])
plt.yticks([])
plt.title('Annotated Mask', fontsize=20)
plt.show()
plt.savefig('test.jpeg')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


In [44]:
cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [ ]:
annotation['contours']

[]

**IoU**

**PA**

In [21]:
from time import time

start = time()

image_id = 0

epsilon = 0.005

coco_pa_ious = []

for i in range(100):
    id_ = coco_annotations[i]['image_id']
    name = coco_data.loadImgs(id_)[0]['file_name']
    image = coco_data.annToMask(coco_annotations[id_])
    
    original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])

    im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

    annotation = coco.annotate(im, do_display=False, do_save=False, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.POLY_APPROX)

    contours = annotation['contours']

    annotated_image = np.zeros((annotation['height'], annotation['width'], 3), dtype=np.uint8)
    cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)

    image_id += 1
    coco_pa_ious.append([name, iou(annotation)])

end = time()
print(f'PA time: {end - start}')


 Annotating image:  000000558840.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000495357.jpg
------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000116061.jpg
--------

In [23]:
# Plot bar chart of ious, image on y axis, iou on x axis
plt.rcParams['figure.figsize'] = [10, 35]
plt.figure()

# sort ious descending
# ious.sort(key=lambda x: x[1], reverse=True)
plt.barh(np.arange(len(coco_pa_ious)), [x[1] for x in coco_pa_ious], align='center', color='slategrey')

# print values on bars on the right
for i, v in enumerate([x[1] for x in coco_pa_ious]):
    plt.text(v + 0.01, i - 0.2, str(round(v, 3)), color='k', fontsize=10)

# add image names to y axis
plt.yticks(np.arange(len(coco_pa_ious)), [x[0] for x in coco_pa_ious], fontsize=10)

# plot vertical line at average and median
plt.axvline(x=np.mean([x[1] for x in coco_pa_ious]), color='r', linestyle='--', linewidth=2)
plt.axvline(x=np.median([x[1] for x in coco_pa_ious]), color='b', linestyle='--', linewidth=2)

# add legend
average_legend = 'Average IoU (' + str(np.round(np.mean([x[1] for x in coco_pa_ious]), 3)) + ')'
median_legend = 'Median IoU (' + str(np.round(np.median([x[1] for x in coco_pa_ious]), 3)) + ')'
plt.legend([average_legend, median_legend], loc='upper right', fontsize=12)

plt.xlabel('IoU', fontsize=20)
plt.ylabel('Image', fontsize=20)
plt.title('IoU per Image (Polygon Approximation - COCO)', fontsize=20, fontweight='bold')

plt.show()
plt.savefig('IoU_PA.jpeg')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


**KMC**

In [9]:
image_id = 0

epsilon = 0.005

coco_kmc_ious = []

for image_id in range(100):
    id_ = coco_annotations[image_id]['image_id']
    name = coco_data.loadImgs(id_)[0]['file_name']
    image = coco_data.annToMask(coco_annotations[id_])
    
    original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])

    im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

    annotation = coco.annotate(im, do_display=False, do_save=True, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.KMC)

    contours = annotation['contours']

    annotated_image = np.zeros((annotation['height'], annotation['width'], 3), dtype=np.uint8)
    cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)

    image_id += 1
    coco_kmc_ious.append([name, iou(annotation)])


 Annotating image:  000000558840.jpg
 Succesfully saved image:  000000558840.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
 Succesfully saved image:  000000200365.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
 Succesfully saved image:  000000200365.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
 Succesfully saved image:  000000200365.jpg 


------------------------------------------------------------------------------------------------------------------------

 Annotating image:  000000200365.jpg
 Succesfully saved image:  000000200365.jpg 


-------------------------------------------------------------------------------------------

In [11]:
# Plot bar chart of ious, image on y axis, iou on x axis
plt.rcParams['figure.figsize'] = [10, 35]
plt.figure()

# sort ious descending
# ious.sort(key=lambda x: x[1], reverse=True)
plt.barh(np.arange(len(coco_kmc_ious)), [x[1] for x in coco_kmc_ious], align='center', color='slategrey')

# print values on bars on the right
for i, v in enumerate([x[1] for x in coco_kmc_ious]):
    plt.text(v + 0.01, i - 0.2, str(round(v, 3)), color='k', fontsize=10)

# add image names to y axis
plt.yticks(np.arange(len(coco_kmc_ious)), [x[0] for x in coco_kmc_ious], fontsize=10)

# plot vertical line at average and median
plt.axvline(x=np.mean([x[1] for x in coco_kmc_ious]), color='r', linestyle='--', linewidth=2)
plt.axvline(x=np.median([x[1] for x in coco_kmc_ious]), color='b', linestyle='--', linewidth=2)

# add legend
average_legend = 'Average IoU (' + str(np.round(np.mean([x[1] for x in coco_kmc_ious]), 3)) + ')'
median_legend = 'Median IoU (' + str(np.round(np.median([x[1] for x in coco_kmc_ious]), 3)) + ')'
plt.legend([average_legend, median_legend], loc='upper right', fontsize=12)

plt.xlabel('IoU', fontsize=20)
plt.ylabel('Image', fontsize=20)
plt.title('IoU per Image (K-Means Clustering - COCO)', fontsize=20, fontweight='bold')

plt.show()
plt.savefig('IoU_KMC.jpeg')

c:\Users\matth\anaconda3\envs\coco\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


**SAM**

In [28]:
# Dependencies required for segment-anything
# %pip install git+https://github.com/facebookresearch/segment-anything.git
# %pip install torch torchvision
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor
import torch
torch.cuda.empty_cache()
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Segment-anything model
# Downloading model checkpoint:
# Utilise the following link: https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
# Place the downloaded checkpoint in a new directory named Sam_checkpoints
sam_checkpoint = "Sam_checkpoints/sam_vit_b_01ec64.pth"
model_type = "vit_b"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_batch=16
)

In [29]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

In [60]:
masks = {}

from time import time
start = time()

for i in range(100):
    id_ = coco_annotations[i]['image_id']
    name = coco_data.loadImgs(id_)[0]['file_name']

    if os.path.exists(f'Evaluation/COCO/segment-anything/{name[:-4]}.jpeg'):
        continue
    

    image = coco_data.annToMask(coco_annotations[id_])
    
    original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])

    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    mask = mask_generator.generate(image)

    sorted_anns = sorted(mask, key=(lambda x: x['area']), reverse=True)
    img = np.zeros((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))

    for ann in sorted_anns:
        # skip first anotation
        if ann is sorted_anns[0]:
            continue
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = 1

    img[:, :, 3] = 1

    # save
    plt.imsave(f'Evaluation/COCO/segment-anything/{name[:-4]}.jpeg', img)

    masks[name] = mask
    # masks.append(mask)
    print(f'SAM Mask generated for {name} -- Time elapsed: {time() - start}')

end = time()
print(f'Total time elapsed: {end - start}')


SAM Mask generated for 000000558840.jpg -- Time elapsed: 9.860119104385376
SAM Mask generated for 000000200365.jpg -- Time elapsed: 18.354334115982056
SAM Mask generated for 000000212545.jpg -- Time elapsed: 29.658732175827026
SAM Mask generated for 000000514915.jpg -- Time elapsed: 40.72724986076355
SAM Mask generated for 000000154589.jpg -- Time elapsed: 51.86639928817749
SAM Mask generated for 000000471175.jpg -- Time elapsed: 60.89506387710571
SAM Mask generated for 000000225919.jpg -- Time elapsed: 70.69145250320435
SAM Mask generated for 000000400728.jpg -- Time elapsed: 80.47364854812622
SAM Mask generated for 000000194306.jpg -- Time elapsed: 89.7560977935791
SAM Mask generated for 000000383780.jpg -- Time elapsed: 98.8330729007721
SAM Mask generated for 000000580255.jpg -- Time elapsed: 108.47515845298767
SAM Mask generated for 000000370210.jpg -- Time elapsed: 116.2275869846344
SAM Mask generated for 000000075283.jpg -- Time elapsed: 124.8877022266388
SAM Mask generated for 0

In [61]:
masks = {}

coco_sam_ious = []

for i in range(100):
    id_ = coco_annotations[i]['image_id']
    name = coco_data.loadImgs(id_)[0]['file_name']

    if os.path.exists(f'Evaluation/COCO/segment-anything/{name[:-4]}.jpeg'):
        image = coco_data.annToMask(coco_annotations[id_])
        
        original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])

        annotation = cv2.imread('./Evaluation/COCO/segment-anything/' + name[:-4] + '.jpeg')
        plt.imshow(annotation)
        annotation = cv2.cvtColor(annotation, cv2.COLOR_BGR2GRAY)
            
        intersection = np.logical_and(image, annotation)
        union = np.logical_or(image, annotation)
        iou_score = np.sum(intersection) / np.sum(union)

        coco_sam_ious.append([name, iou_score])


In [62]:
# Plot bar chart of ious, image on y axis, iou on x axis
plt.rcParams['figure.figsize'] = [10, 35]
plt.figure()

# sort ious descending
# ious.sort(key=lambda x: x[1], reverse=True)
plt.barh(np.arange(len(coco_sam_ious)), [x[1] for x in coco_sam_ious], align='center', color='slategrey')

# print values on bars on the right
for i, v in enumerate([x[1] for x in coco_pa_ious]):
    plt.text(v + 0.01, i - 0.2, str(round(v, 3)), color='k', fontsize=10)

# add image names to y axis
plt.yticks(np.arange(len(coco_sam_ious)), [x[0] for x in coco_sam_ious], fontsize=10)

# plot vertical line at average and median
plt.axvline(x=np.mean([x[1] for x in coco_sam_ious]), color='r', linestyle='--', linewidth=2)
plt.axvline(x=np.median([x[1] for x in coco_sam_ious]), color='b', linestyle='--', linewidth=2)

# add legend
average_legend = 'Average IoU (' + str(np.round(np.mean([x[1] for x in coco_sam_ious]), 3)) + ')'
median_legend = 'Median IoU (' + str(np.round(np.median([x[1] for x in coco_sam_ious]), 3)) + ')'
plt.legend([average_legend, median_legend], loc='upper right', fontsize=12)

plt.xlabel('IoU', fontsize=20)
plt.ylabel('Image', fontsize=20)
plt.title('IoU per Image (Segment Anything - COCO)', fontsize=20, fontweight='bold')

plt.show()
plt.savefig('COCO_IoU_SAM.jpeg')

c:\Programming\Python\Environments\anaconda\envs\coco\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


**Compactness**

In [1]:
def compactness(annotation):
    # area of annotation
    area = annotation['image'].shape[0] * annotation['image'].shape[1]

    # perimeter of annotation
    perimeter = 0
    for contour in annotation['contours']:
        perimeter += cv2.arcLength(contour, True)

    return (perimeter ** 2) / area

**PA**

In [ ]:
from time import time

start = time()

image_id = 0

epsilon = 0.005

coco_pa_compactness = []

for image_id in range(100):
    id_ = coco_annotations[image_id]['image_id']
    name = coco_data.loadImgs(id_)[0]['file_name']
    image = coco_data.annToMask(coco_annotations[id_])
    
    original = skimage.io.imread(coco_data.loadImgs(id_)[0]['coco_url'])

    im = (id_, name, image, 'COCOEvaluation', 'COCO', './Evaluation/COCO/')

    annotation = coco.annotate(im, do_display=False, do_save=True, do_cvt=False, annotation_color=coco_color, epsilon=0.005, configuration=coco.PA)

    contours = annotation['contours']

    annotated_image = np.zeros((annotation['height'], annotation['width'], 3), dtype=np.uint8)
    cv2.drawContours(annotated_image, annotation['contours'], -1, (255, 0, 0), 3, cv2.LINE_AA)

    image_id += 1
    coco_pa_ious.append([name, iou(annotation)])

end = time()
print(f'PA time: {end - start}')